# Manhattan Plot

In this notebook we intend to show how can be used the library `IDEAL-GENOM` to generate a Miami Plot. 

In [ ]:
import sys
import os

import pandas as pd

import ipywidgets as widgets
from IPython.display import display

# add parent directory to path
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

from ideal_genom.manhattan_type import manhattan_draw

At the end of the next cell, please fill the required values. The notebook could be modified according to the user interest and particular requirements, nevertheless in order to use as it is, the user should have the following specifications in mind. It is expected that all files are tab-separated-values, but the user can change this easily on the corresponding cell.

1. `input_path` is the path to the folder where the necessary files to draw the plot are.
2. `input_name` is the name of the file containing the **GWAS** summary statistics.
3. `highlights` is the name of the file containing the top hits to highlight.
4. `annotate` is the name of the file containing the rsID of the SNPs to annotate.

Usually one annotate a subset of the highlighted SNPs, but we give the freedom to annotate a different set of SNPs.

In [ ]:
# Create interactive widgets for input
input_path = widgets.Text(
    value='/home/luis/data/gwasResult/',
    description='Path to project folder:',
    style={'description_width': 'initial'}
)

input_name = widgets.Text(
    value='annotated_normalized_combined_R2_0.3.dose_step2_sex_pheno-glm.PHENO1.glm.logistic.hybrid',
    description='Name of GWAS summary file:',
    style={'description_width': 'initial'}
)

highlight = widgets.Text(
    value='table_lead_SNPS_GWAS_glm_logistic_final_paper',
    description='Name of file with SNPs to highlight:',
    style={'description_width': 'initial'}
)

annotate = widgets.Text(
    value='table_lead_SNPS_GWAS_glm_logistic_final_paper',
    description='Name of file with SNPs to annotate:',
    style={'description_width': 'initial'}
)

# Display the widgets
display(input_path, input_name, highlight, annotate)

# Function to get the text parameter values
def get_params():
    return input_path.value, input_name.value, highlight.value, annotate.value

In [ ]:
# Use the parameter values
path_params = get_params()
print(f"Parameter 1: {path_params[0]}")
print(f"Parameter 2: {path_params[1]}")
print(f"Parameter 3: {path_params[2]}")
print(f"Parameter 4: {path_params[3]}")

Please, provide the columns to load from the **GWAS** summary statistics. The essential columns are those that contains chromosome, base-pair position, rsID and p-value. It is important to remark that columns names most coincide in both files, at least for the four columns that will be used to generate the Manhattan plot.

In [ ]:
cols_touse = widgets.Textarea(
    value="#CHROM, POS, ID, P",
    description='Columns to use on the Mannhattan plot (comma-separated):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
display(cols_touse)

def get_cols():
    return cols_touse.value

In [ ]:
cols = get_cols()

df_gwas = pd.read_csv(
    os.path.join(path_params[0], path_params[1]), sep='\t', usecols=[col.strip() for col in cols.split(',')]
)
df_gwas.head(5)

In [ ]:
rsID = widgets.Text(
    value='SNP',
    description='Column name with the rsID:',
    style={'description_width': 'initial'}
)

gene_col = widgets.Text(
    value='Nearest Gene',
    description='Column name with the nearest gene name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display(rsID, gene_col)

def get_rsID():
    return rsID.value
def get_gene_col():
    return gene_col.value

In [ ]:
gwas_rsID = get_rsID()

if path_params[2] is not None or path_params[2] != '':
    highlit_path = os.path.join(path_params[0], path_params[2])
    if os.path.exists(highlit_path):
        df_high = pd.read_csv(
            highlit_path, sep='\t'
        )
        to_highlight = df_high[gwas_rsID].to_list()
        print(to_highlight[:10])
        del df_high
    else:
        print('Path to file with SNPs to highlight does not exist')
        to_highlight = []
else:
    print('No file with SNPs to highlight')
    to_highlight = []

In [ ]:
gene_col = get_gene_col()

if path_params[3] is not None and path_params[3] != '':
    anno_path = os.path.join(path_params[0], path_params[3])
    if os.path.exists(anno_path):
        df_anno = pd.read_csv(
            anno_path, sep='\t',
        )
        if gene_col not in df_anno.columns:
            to_annotate = df_anno[[gwas_rsID]]
        else:
            to_annotate = df_anno[[gwas_rsID, gene_col]].copy()
        del df_anno
    else:
        print('Path to file with SNPs to annotate does not exist')
        to_annotate = pd.DataFrame()
else:
    print('No file with SNPs to annotate')
    to_annotate = pd.DataFrame()

to_annotate.head()

In [ ]:
SNP_col = widgets.Text(
    value='ID',
    description='Column with rsID:',
    style={'description_width': 'initial'}
)

CHR_col = widgets.Text(
    value='#CHROM',
    description='Column with chromosome:',
    style={'description_width': 'initial'}
)

POS_col = widgets.Text(
    value='POS',
    description='Column with base-pair position:',
    style={'description_width': 'initial'}
)

P_col = widgets.Text(
    value='P',
    description='Column with p-values:',
    style={'description_width': 'initial'}
)
display(SNP_col, CHR_col, POS_col, P_col)

def get_col_names():
    return SNP_col.value, CHR_col.value, POS_col.value, P_col.value

In [ ]:
snp_col, chr_col, pos_col, p_col = get_col_names()
to_annotate = to_annotate.rename(columns={gwas_rsID: snp_col, gene_col: 'GENENAME'})

In [ ]:
manhattan_draw(
    data_df     =df_gwas,
    chr_col     =chr_col,
    pos_col     =pos_col,
    p_col       =p_col,
    snp_col     =snp_col,
    plot_dir    =path_params[0],
    to_highlight=to_highlight,
    to_annotate =to_annotate,
    save_name   ='manhattan_plot_indian_gwas.jpeg',
    gen_col     =gene_col
)